In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
%matplotlib inline

In [131]:
url = 'https://en.wikipedia.org/wiki/'
r = requests.get(url + 'Afghanistan')

soup = BeautifulSoup(r.text,'html.parser')
soup = soup.find('table', {'class': 'infobox geography vcard'})
tr_tags = soup.find_all('tr')

In [135]:
country_languages_dict = dict()

for tag in tr_tags:
    if tag.find('a', {'href': '/wiki/Official_language'}) != None:
        
        country_languages = soup.find('div', {'class': 'hlist hlist-separated'})
        country_languages_list = []

        for lang in country_languages.find_all('li'):
            country_languages_list.append(lang.text)

        country_languages_dict.update({'Afghanistan': country_languages_list})

        country_languages_dict
        

country_languages_dict

{'Afghanistan': ['Pashto', 'Dari[1]']}

In [122]:
country_religion_dict = dict()

for tag in tr_tags:
    if tag.find('a', {'href': '/wiki/Religion_in_' + 'Afghanistan'}) != None:
        
        religion = soup.find('a', {'href': '/wiki/Religion_in_' + 'Afghanistan'}).text
        
        
        country_religion_dict.update({'Afghanistan': religion})
    else:
        religion
religion

'Islam'

Take dataset from https://github.com/mledoze/countries

In [147]:
countries = pd.read_json('countries.json')
cols = ['area', 'cca2', 'cca3', 'ccn3', 'languages', 'name']
countries = countries[cols]

countries.name[0]['official']

'Aruba'

Found dataset with all country codes https://github.com/datasets/country-codes/blob/master/README.md

In [104]:
cols = ['official_name_en', 'ISO3166-1-Alpha-2', 'ISO3166-1-Alpha-3', 'ISO3166-1-numeric', \
        'Continent', 'Region Name', 'Region Code']
codes = codes[cols]
codes.head()

,official_name_en,ISO3166-1-Alpha-2,ISO3166-1-Alpha-3,ISO3166-1-numeric,Continent,Region Name,Region Code
1,Afghanistan,AF,AFG,4.0,AS,Asia,142.0
2,Albania,AL,ALB,8.0,EU,Europe,150.0
3,Algeria,DZ,DZA,12.0,AF,Africa,2.0
4,American Samoa,AS,ASM,16.0,OC,Oceania,9.0
5,Andorra,AD,AND,20.0,EU,Europe,150.0
